In [1]:
from langchain_core.tools import tool

In [2]:
# s1 - create a function
def multiply(a, b):
    '''Multiplies two numbers together.'''
    return a*b

In [3]:
# s2 - add type hints (type hinting)
def multiply(a: int, b: int) -> int:
    '''Multiplies two numbers together.'''
    return a*b

In [4]:
# s3 - add tool decorator

"""
    @tool - decorator is used to mark this function as a special function.
"""
@tool
def multiply(a: int, b: int) -> int:
    '''Multiplies two numbers together.'''
    return a*b

In [7]:
ans = multiply.invoke({'a':5, 'b':8})
print(ans)

40


In [9]:
# properties

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiplies two numbers together.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
